# **Academic Title Generator**

## Generate believable academic titles from a paper abstract.

This code allows you to input academic abstracts in order to generate academic paper or poster titles.




# Install Transformers Library and the Tokenization Library

**Abstractive text summarization- T5 Keras**

Install the pre-trained transformer from the Keras library as well as the tokenizaton library in the pre-trained model to fine-tune that model.

Hugging face T5 pretrained model

In [ ]:
!pip install transformers --quiet
!pip install sentencepiece --quiet
import tensorflow as tf
import numpy as np
import re
import string
import pandas as pd
import tensorflow_datasets as tfds
from transformers import T5Tokenizer,T5ForConditionalGeneration

     |████████████████████████████████| 4.4 MB 4.8 MB/s 
     |████████████████████████████████| 596 kB 65.3 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 6.6 MB 68.6 MB/s 
     |████████████████████████████████| 1.2 MB 15.1 MB/s 


# Download the Pretrained Model 

In [ ]:
!curl -L -o 'title_generation_model_5epochs.zip' 'https://drive.google.com/u/0/uc?id=1lxwBfFbzUYyDCGEEWqGwQbA5D60pob5M&export=download&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  786M  100  786M    0     0  95.8M      0  0:00:08  0:00:08 --:--:--  120M


# Unzip Pretrained Model into Colab Drive

In [ ]:
!unzip "title_generation_model_5epochs.zip"

Archive:  title_generation_model_5epochs.zip
   creating: title_generation_model/
  inflating: title_generation_model/config.json  
  inflating: title_generation_model/pytorch_model.bin  


#Load the Tokenizer
**T5 Tokenizer and Config**

The tokenizer breaks text down into represenations called tokens that allow the encoder to effectively represent the text in a semantic and structural manner.

In [ ]:
# Bert Tokenizer (leverages SentencePiece and Unicode Normalizaiton)
tokenizer = T5Tokenizer.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


# Import the Neural Network Libraries

These enable the model to be trained on new text and encode that text in a recurrent neural network.

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
model = T5ForConditionalGeneration.from_pretrained("title_generation_model")
model = model.to(device)

# Define a Function to Get Titles from the Input Abstract Text
**Test Model using a Beam Search Algorithm**

In [ ]:
import torch
def getSummary_greedy(text, number_examples = 10):
  with torch.no_grad():
    input_ids = tokenizer("summarize: "+text, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    outputs = model.generate(input_ids,do_sample =False,min_length=3,max_length=20,repetition_penalty=1.0)
    decodedwords = tokenizer.decode(outputs[0])
    return decodedwords

In [ ]:
import torch
def getSummary_probability(text, number_examples = 10):
  with torch.no_grad():
    input_ids = tokenizer("summarize: "+text, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    outputs = model.generate(input_ids,do_sample =True,num_beams=1,min_length=3,max_length=20,repetition_penalty=1.0)
    decodedwords = tokenizer.decode(outputs[0])
    return decodedwords

In [ ]:
import torch
def getSummary_beamsearch(text, number_examples = 10):
  with torch.no_grad():
    input_ids = tokenizer("summarize: "+text, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    outputs = model.generate(input_ids,num_return_sequences = number_examples,do_sample =True,num_beams=3,min_length=3,max_length=30,repetition_penalty=1.0)
    decodedwords = tokenizer.decode(outputs[0])
    return decodedwords

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')

# Test the Model on a Sample Academic Abstract

In [ ]:
abstract ="""This research investigates how the perceived balance between cognition and emotion in others contributes to fear of other agents – “scary minds”.  Specifically, we explore the hypothesis that a threatening target with an unbalanced mind—asymmetric capacities for cognition (e.g., self-control and reasoning) and emotion (e.g., sensations and emotions)—is associated with greater fear of that target.  In four studies, targets with a perceived imbalance between capacities for cognition and emotion were rated as more frightening than those with relatively more matched levels of cognition and emotion.  These effects were observed using samples of scary monsters (Studies 1 & 2), scary animals (Studies 2 & 3),  and infected humans (Study 4).  We also find experimental evidence that imbalanced targets are perceived as less controllable, which mediates the effect of imbalance on fear.  These results suggest that the perceived degree of mismatch between cognition and experience predicts fear among scary targets, and highlight mind perception as an important part of the appraisal process of emotion."""

In [ ]:
getSummary_greedy(abstract)

'<pad> Unbalanced Mind and Fear of Scary Minds: A Multilevel Investigation</s>'

In [ ]:
getSummary_probability(abstract)

'<pad> Emotion and Fear of Scary Minds</s>'

In [ ]:
getSummary_beamsearch(abstract)

'<pad> Unbalanced Mind and Fear of Scary Minds: The Role of Mind Perception</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

# Generate Many Titles at Once

In [ ]:
for i in range(100):
  generated_title = getSummary_probability(abstract)
  print(generated_title)

<pad> Unbalanced Mind and Fear among Scary Mentalities: The Mediating Role of
<pad> Scale-Level Balance at Risk: Interactions With Scary Minds</s>
<pad> Underbalancing Mind and Psychological Behaviors: An Involuntary Measure of Perception
<pad> Consideration and Measurement of Scary Minds</s>
<pad> Fear of Fear Among Scary Minds: Its Corollary to Brain Assessment</s>
<pad> Fear at Scary Mental Targets: Using Emotional and Cognitive Context</s>
<pad> Unbalanced Mind and Fear of Scary Minds</s>
<pad> The Effect of Inbalancing Mind and Emotion on Physical Fear Among Scary Minds
<pad> Negative Assessment of Scary Minds on Fear</s>
<pad> Mismatch between Cognition and Emotions to Predict Scary Minds</s>
<pad> Constructive Effects of Mind Balance on Scary Minds</s>
<pad> The Debalance Between Cognitive and Emotion</s>
<pad> Scary Minds as a Mediator of Perceived Balance</s>
<pad> a Mind of Unbalanced Characteristics and Potential Behavioral Effects on Be
<pad> Fear, Negative Cognition, and E